# सेमॅंटिक कर्नल

या कोड नमुन्यात, तुम्ही [सेमॅंटिक कर्नल](https://aka.ms/ai-agents-beginners/semantic-kernel) AI फ्रेमवर्क वापरून एक साधा एजंट तयार कराल.

या नमुन्याचा उद्देश तुम्हाला पुढील कोड नमुन्यांमध्ये वेगवेगळ्या एजंटिक पॅटर्न्स अंमलात आणताना वापरण्यात येणाऱ्या टप्प्यांचे प्रदर्शन करणे आहे.


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## क्लायंट तयार करणे

या उदाहरणात, आपण [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) वापरून LLM साठी प्रवेश मिळवणार आहोत.

`ai_model_id` म्हणून `gpt-4o-mini` परिभाषित केले आहे. GitHub Models मार्केटप्लेसमध्ये उपलब्ध असलेल्या इतर मॉडेल्समध्ये बदल करून वेगवेगळे परिणाम पाहण्याचा प्रयत्न करा.

`Azure Inference SDK` वापरण्यासाठी, जो GitHub Models साठी `base_url` वापरतो, आपण Semantic Kernel मध्ये `OpenAIChatCompletion` कनेक्टर वापरणार आहोत. Semantic Kernel वापरण्यासाठी इतर मॉडेल प्रदात्यांसाठी [उपलब्ध कनेक्टर्स](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) देखील आहेत.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## एजंट तयार करणे

खाली आपण `TravelAgent` नावाचा एजंट तयार करतो.

या उदाहरणासाठी, आम्ही अतिशय सोप्या सूचना वापरत आहोत. तुम्ही या सूचनांमध्ये बदल करू शकता आणि पाहू शकता की एजंट वेगळ्या प्रकारे कसा प्रतिसाद देतो.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## एजंट चालवणे

आता आपण `ChatHistoryAgentThread` प्रकाराचा थ्रेड परिभाषित करून एजंट चालवू शकतो. कोणत्याही आवश्यक प्रणाली संदेशांना एजंटच्या invoke_stream च्या `messages` कीवर्ड आर्ग्युमेंटमध्ये प्रदान केले जाते.

हे परिभाषित केल्यानंतर, आपण `user_inputs` तयार करतो, जे वापरकर्ता एजंटला पाठवत असतो. या प्रकरणात, आपण हा संदेश `Plan me a sunny vacation` असा सेट केला आहे.

एजंट वेगवेगळ्या प्रकारे कसा प्रतिसाद देतो हे पाहण्यासाठी हा संदेश बदलण्यास मोकळे रहा.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
